In [12]:
import json

def parse_json_config(file_path):
    with open(file_path, 'r') as file:
        config = json.load(file)
    return config

# Example usage
config = parse_json_config('MLP.json')
print(config)

{'layers': [{'type': 'input', 'neurons': 4}, {'type': 'hidden', 'neurons': 4, 'activation': 'sigmoid'}, {'type': 'output', 'neurons': 2, 'activation': 'sigmoid'}], 'learning_rate': 0.1, 'epochs': 10000, 'batch_size': 32}


In [10]:
import numpy as np
import json

class MLP:
    def __init__(self, config_path):
        # Load configuration from JSON file
        with open(config_path, 'r') as file:
            config = json.load(file)
        
        self.layers_config = config.get('layers', [])
        self.learning_rate = config.get('learning_rate', 0.01)
        self.epochs = config.get('epochs', 1000)
        self.batch_size = config.get('batch_size', 32)  # Not used in this implementation, but read in
        
        self.input_size = None
        self.initialize_network()
    
    def validate_config(self):
        """ Validate the configuration parameters """
        if not self.layers_config:
            raise ValueError("The 'layers' configuration is missing or empty.")
        
        layer_types = {'input', 'hidden', 'output'}
        activations = {'relu', 'sigmoid', 'tanh'}
        
        for layer in self.layers_config:
            layer_type = layer.get('type')
            if layer_type not in layer_types:
                raise ValueError(f"Invalid layer type: {layer_type}")
            
            if 'neurons' not in layer:
                raise ValueError(f"Layer type '{layer_type}' is missing the 'neurons' key.")
            
            if layer_type != 'input' and 'activation' not in layer:
                raise ValueError(f"Layer type '{layer_type}' requires an 'activation' key.")
            
            if layer_type != 'input':
                activation = layer.get('activation')
                if activation not in activations:
                    raise ValueError(f"Invalid activation function: {activation}")
    
    def initialize_network(self):
        self.validate_config()
        
        self.weights = []
        self.biases = []
        layer_input_size = None
        
        for layer in self.layers_config:
            layer_type = layer['type']
            layer_size = layer['neurons']
            
            if layer_type == 'input':
                self.input_size = layer_size
                layer_input_size = self.input_size
                continue
            
            if layer_input_size is None:
                raise ValueError("Input layer size not defined.")
            
            self.weights.append(np.random.randn(layer_input_size, layer_size))
            self.biases.append(np.random.randn(1, layer_size))
            layer_input_size = layer_size
        
        # Debug prints to verify initialization
        print("Weights:", self.weights)
        print("Biases:", self.biases)
    
    def activation(self, x, func):
        if func == 'relu':
            return np.maximum(0, x)
        elif func == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif func == 'tanh':
            return np.tanh(x)
        else:
            raise ValueError(f"Unknown activation function: {func}")
    
    def activation_derivative(self, x, func):
        if func == 'relu':
            return np.where(x > 0, 1, 0)
        elif func == 'sigmoid':
            return x * (1 - x)
        elif func == 'tanh':
            return 1 - x ** 2
        else:
            raise ValueError(f"Unknown activation function: {func}")
    
    def forward_propagation(self, X):
        self.z = []
        self.a = [X]
        
        for i in range(len(self.layers_config) - 1):
            layer = self.layers_config[i + 1]  # Skip input layer
            layer_activation = layer.get('activation', None)
            
            z = np.dot(self.a[-1], self.weights[i]) + self.biases[i]
            self.z.append(z)
            a = self.activation(z, layer_activation)
            self.a.append(a)
        
        return self.a[-1]
    
    def backward_propagation(self, X, y):
        m = y.shape[0]
        delta = self.a[-1] - y
        d_weights = []
        d_biases = []

        for i in reversed(range(len(self.weights))):
            d_weights.insert(0, np.dot(self.a[i].T, delta) / m)
            d_biases.insert(0, np.sum(delta, axis=0, keepdims=True) / m)
            if i != 0:
                delta = np.dot(delta, self.weights[i].T) * self.activation_derivative(self.a[i], self.layers_config[i]['activation'])
        
        return d_weights, d_biases
    
    def update_weights(self, d_weights, d_biases):
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * d_weights[i]
            self.biases[i] -= self.learning_rate * d_biases[i]
    
    def train(self, X, y):
        for epoch in range(self.epochs):
            output = self.forward_propagation(X)
            d_weights, d_biases = self.backward_propagation(X, y)
            self.update_weights(d_weights, d_biases)
            if epoch % 1000 == 0:
                loss = np.mean((output - y) ** 2)
                print(f"Epoch {epoch}, Loss: {loss}")
    
    def predict(self, X):
        return self.forward_propagation(X)

# Example usage:
if __name__ == "__main__":
    config_path = 'D:\\BTech\\sem 7\\NNDL\\lab\\mlp.json'
    
    X = np.array([[0, 0, 0, 0], [0, 1, 0, 1], [1, 0, 1, 0], [1, 1, 1, 1]])  # Example input
    y = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # Example output for 2 neurons in output layer

    mlp = MLP(config_path)
    mlp.train(X, y)
    predictions = mlp.predict(X)
    print("Predictions:", predictions)


Weights: [array([[ 0.89240733,  1.15232528, -1.58683502,  1.08868314],
       [-0.53985535, -0.21466758,  1.93228834,  0.71542268],
       [ 0.25862468,  0.56310304, -1.4396527 , -2.14956421],
       [ 1.67937681,  0.86758745, -0.54820952,  1.4371271 ]]), array([[ 0.48588327, -0.61967597],
       [ 0.17659387,  0.89786977],
       [ 0.04809907,  0.61024777],
       [-0.64826161,  0.39791691]])]
Biases: [array([[ 1.33530157,  1.08049492,  0.27966992, -0.89818769]]), array([[ 0.10701393, -0.83115166]])]
Epoch 0, Loss: 0.2295572954371205
Epoch 1000, Loss: 0.002873509364470388
Epoch 2000, Loss: 0.00042090153136964013
Epoch 3000, Loss: 0.0001469746313982441
Epoch 4000, Loss: 7.242176292456313e-05
Epoch 5000, Loss: 4.2628700146626425e-05
Epoch 6000, Loss: 2.7929234041947914e-05
Epoch 7000, Loss: 1.964933757443239e-05
Epoch 8000, Loss: 1.454296470684659e-05
Epoch 9000, Loss: 1.117944661883051e-05
Predictions: [[0.00314134 0.00308356]
 [0.00203839 0.99730251]
 [0.99803151 0.00398472]
 [0.99666